In [1]:
import pandas as pd
from statistics import mean
from nltk.tokenize import word_tokenize

In [2]:
import numpy as np

In [3]:
#imports the data
# roughDat= pd.read_csv("PlayDat.csv")
roughDat= pd.read_csv("../../data/raw/Emergent_NAACL2016/emergent/url-versions-2015-06-14-clean.csv")

In [4]:
#removes unnecessary columns.
# roughDat = roughDat.drop(columns=['Unnamed: 0', 'articleHeadline', 'claimId',
#        'articleId', 'articleHeadlineStance'])

In [5]:
# import os
# os.environ["CORENLP_HOME"] = r'D:\\Learning Material\\IR\\stanford-corenlp-full-2018-10-05\\stanford-corenlp-full-2018-10-05'

In [4]:
#import re 
mylist=roughDat["articleHeadline"].tolist()
#myshortlist = [re.sub(r'[^\w\s\']','',s) for s in mylist] for removing punctuation. 

In [5]:
from stanfordnlp.server import CoreNLPClient
rootIndex =[]
rootDistances = []
with CoreNLPClient(annotators=['depparse'], timeout=60000, memory='16G') as client:
    # submit the request to the server
    for sentences in mylist:

        ann = client.annotate(sentences)

        # get the first sentence
        sentence = ann.sentence[0]
        # get the dependency parse of the first sentence
        dependency_parse = sentence.basicDependencies
        root=int(dependency_parse.root[0])
        rootIndex.append(root)
        neg_Pos=None
        for i in dependency_parse.edge:
            if i.dep=="neg":
                neg_Pos=int(i.target)
        if neg_Pos!=None:
            rootDist=abs(root - neg_Pos)
        else:
            rootDist=0
        rootDistances.append(rootDist)        

Starting server with command: java -Xmx16G -cp /Users/nikilsaldanaha/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-88fbde6bba0a4438.props -preload depparse


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
refuting =['fake',
           'fraud',
           'hoax',
           'false',
           'deny', 
           'denies',
           'refute',
           'not',
           'despite',
           'nope',
           'doubt', 
           'doubts',
           'bogus',
           'debunk',
           'pranks',
           'prank',
           'retract']
hedging = [
        'alleged', 'allegedly',
        'apparently',
        'appear', 'appears',
        'claim', 'claims',
        'could',
        'evidently',
        'largely',
        'likely',
        'mainly',
        'may', 'maybe', 'might',
        'mostly',
        'perhaps',
        'presumably',
        'probably',
        'purported', 'purportedly',
        'reported', 'reportedly',
        'rumor', 'rumour', 'rumors', 'rumours', 'rumored', 'rumoured',
        'says',
        'seem',
        'somewhat',
        'supposedly',
        'unconfirmed']

In [ ]:
mytokenizedlist =[word_tokenize(i) for i in mylist]

In [ ]:
hedged_RDist = []
k = 0
for i in mytokenizedlist:
    Distances = []
    for j in hedging:
        if j in i:
            Dist = abs(rootIndex[k]-i.index(j))
            Distances.append(Dist)
        else:
            Distances.append(0)
    k+=1
    Distances = [i for i in Distances if i != 0]
    if len(Distances)==0:
        Distances.append(0)
    hedged_RDist.append(min(Distances))

In [ ]:
refute_RDist =[]
k=0
for i in mytokenizedlist:
    Distances=[]
    for j in refuting:
        if j in i:
            Dist=(abs(rootIndex[k]-i.index(j)))
            Distances.append(Dist)
        else:
            Distances.append(0)
    k+=1
    Distances = [i for i in Distances if i != 0]
    if len(Distances)==0:
        Distances.append(0)
    refute_RDist.append(min(Distances))

In [ ]:
hedged_RDist = np.array(hedged_RDist)

In [ ]:
refute_RDist = np.array(refute_RDist)

In [ ]:
root_dist_feature_df = pd.DataFrame({"articleId": roughDat["articleId"].tolist(), "rootDistance": rootDistances, "hedgedRootDistance": hedged_RDist, "refuteRootDistance": refute_RDist})



In [ ]:
mindist = root_dist_feature_df.min(axis=1)
root_dist_feature_df_min = pd.DataFrame({"articleId": roughDat["articleId"].tolist(), "minrootDistance": mindist})


In [ ]:
root_dist_feature_df.to_csv("../../data/processed/features/root_dist.csv", index=False)
root_dist_feature_df_min.to_csv("../../data/processed/features/root_dist_min.csv", index=False)
